https://www.ibm.com/cloud/learn/recurrent-neural-networks<br>
https://d2l.ai/chapter_recurrent-neural-networks/rnn.html

RNN stands for Recurrent Neural Networks. RNN is a type of artificial neural network which uses sequential data such as:
an audio clip, a phrase is a sequence of words, DNA, a time series, a sequence of video frames. Sequential data can be seen as series.

We use RNN to solve problems such :
* speech recognition : given an audio clip X, we want to map it to a text transcript Y. Both X and Y are sequence data. Y is a sequence of words. Speech recognition is a sequence to sequence problem. seq2seq. The length of the sequence X differs from the length of the sequence Y. It's a many to many problem.
* machine translation : given an input phrase X we want to ouput the translation Y. The lenght of X and Y could differ. It is a many to many problem 
* name entity recognition : given a sentence X, we want to identify the people Y in that sentence. 
* sentiment classification : given an input phrase X we want to predict if the sentence is positive, negative or neutral. It is a many to one problem.


# SEQUENTIAL DATA NOTATION
Let $(X^{(1)},Y^{(1)}),(X^{(2)},Y^{(2)}),\dots,(X^{(n)},Y^{(n)})$ a training set we can also view as a n-sized batch. Each $X^{(i)}$ is an input sequence. Each $Y^{(i)}$ is a target output sequence.<br>
Different exemple in the training set can have different lengths. For each $i\in \{1,\dots n\}$, $T_{x}^{(i)}$ denots the length of $X^{(i)}$. For each $i\in \{1,\dots n\}$, $T_{y}^{(i)}$ denots the length of $Y^{(i)}$.<br>
Each $X^{(i)}$ is a sequential data indexed by $t$, so $X^{(i)}=X^{(i)<1>},\dots,X^{(i)<t>},\dots,X^{(i)<T_{x}^{(i)}>}$ . $X^{(i)<t>}$ refers to the t th élément of the i th exemple. Each $X^{(i)<t>}\in\mathbb{R}^{d\times1}$ is a vector.<br> 
Each $Y^{(i)}$ is a sequential data indexed by $t$, so $Y^{(i)}=Y^{(i)<1>},\dots,Y^{(i)<t>},\dots,Y^{(i)<T_{y}^{(i)}>}$ . $Y^{(i)<t>}$ refers to the t th élément of the i th exemple. Each $Y^{(i)<t>}\in\mathbb{R}^{q\times1}$ is a vector.

Consider the following sentences and their translations:
* le chien joue -> the dog is playing
* Machiavel était un homme politique italien de la renaissance -> Machiavelli was an Italian Renaissance politician 

In this example, we have a 2-sized batch. 
* $X^{(1)<1>}$ is a vector representation of "le". $X^{(1)<2>}$ is a vector representation of "chien". $X^{(1)<3>}$ is a vector representation of "joue". Then, we have $T_{x}^{(1)}=3$.
* $Y^{(1)<1>}$ is a vector representation of "the". $Y^{(1)<2>}$ is a vector representation of "dog". $Y^{(1)<3>}$ is a vector representation of "is". $Y^{(1)<4>}$ is a vector representation of "playing".Then, we have $T_{y}^{(1)}=4$.
* $X^{(2)<1>}$ is a vector representation of "Machiavel". $X^{(2)<2>}$ is a vector representation of "était". $X^{(2)<3>}$ is a vector representation of " un". $X^{(2)<4>}$ is a vector representation of "homme". $X^{(2)<5>}$ is a vector representation of "politique". $X^{(2)<6>}$ is a vector representation of "italien". $X^{(2)<7>}$ is a vector representation of "de". $X^{(2)<8>}$ is a vector representation of "la". $X^{(2)<9>}$ is a vector representation of "renaissance".Then, we have $T_{x}^{(2)}=9$.
* $Y^{(2)<1>}$ is a vector representation of "Machiavelli". $Y^{(2)<2>}$ is a vector representation of "was". $Y^{(2)<3>}$ is a vector representation of "an". $Y^{(2)<4>}$ is a vector representation of "Italian". $Y^{(2)<5>}$ is a vector representation of "Renaissance". $Y^{(2)<6>}$ is a vector representation of "politician". Then we have $T_{y}^{(2)}=6$.

# RECURRENT NEURAL NETWORKS MODEL
Below, we draw the flow of a typical Recurrent Neural Network.

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/overall_RNN_network..png" width="80%" />
</center>

At least in this example, $T_{x}$ is equal to $T_{y}$. The architecture will change a bit if $T_{x}$ and $T_{y}$ are not identical.<br>
At each time step ($t\in\{1,\dots,T_{x} \}$), we take the input $X^{<t>}$ and feed it into a neural network layer and we try to predict the output $Y^{<t>}$. 

What a RNN does instead of just predicting $Y^{ <t>}$ using only $X^{<t>}$, it also gets to input some information from the previous layer using the hidden state $a^{<t>}$. We see in the diagram that the hidden state $a^{<t-1>}$ from time step t-1 is passed on to time step t. At each time step the RNN passes on his hidden state to the next time step for it to use.<br>
Hidden state at time zero $a^{<0>}$ is usually the vector of zeros. Some researchers initialize $a^{<0>}$ randomly. 

* $w_{ax}$ is governing the connection from $X^{<t>}$ to the hidden layer. 
* We use $w_{ax}$ for every time step. The hidden states $a^{<t>}$ (the horizontal connections) are governed by $w_{aa}$. $w_{aa}$ is used on every time step.
* Similarly, $w_{ya}$ governs the output prediction $\hat{Y}^{<t>}$.  $w_{ya}$ is used on every time step.
A RNN shares his parameters across each layer of the network. RNN share the same weight parameter within each layer of the network. In other words, $w_{ax}, w_{aa}, w_{ya}$ are used on every time step.

**One weakness of this RNN is that it only uses the information that is earlier in the sequence to make a prediction. This RNN does not use information later in the sequence.**<br>
A recurrent neural network can be seen as the repeated use of a single cell. The following figure describes the operations for a single time-step of an RNN cell.

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/Naive_cell_Architecture.png" width="100%" />
</center>

## Forward propagation
**Forward propagation**<br>
for each time step t>0:<br>
$\left\{\begin{array}{l} 
a^{<t>}=g_{a}(w_{ax}X^{<t>}+w_{aa}a^{<t-1>}+b_{a})\\
\hat{Y}^{<t>}=g_{y}(w_{ya}a^{<t>}+b_{y})
\end{array}\right.$<br>
$g_{a}$ is an activation function.$g_{a}$ is often tanh or relu function. $g_{y}$ is also an activation function. $g_{y}$ could be softmax or sigmoid function. In the case of a binary classification problem we $g_{y}$ is the sigmoid activation function. In the case of a k-way classification problem, $g_{y}$ is the softmax activation function. 
* The hidden states $a^{<t>}\in \mathbb{R}^{h\times n}$ and the input $X^{<t>}\in \mathbb{R}^{d\times n}$
* The weight parameter $w_{ax}\in \mathbb{R}^{h\times d}$. The weight parameter $w_{aa}\in \mathbb{R}^{h\times h}$. $b_{a}\in \mathbb{R}^{h\times 1}$ is the bias parameter.
* Th output $\hat{Y}^{<t>}\in \mathbb{R}^{q\times n}$. The weight parameter $w_{ya}\in \mathbb{R}^{q\times h}$. $b_{y}\in \mathbb{R}^{q\times 1}$ is the bias parameter.

We can simplify the notation of the forward propagation:
for each time step t>0:<br>
$\left\{\begin{array}{l} 
a^{<t>}=g_{a}(w_{a}[X^{<t>},a^{<t-1>}]^{t}+b_{a})\\
\hat{Y}^{<t>}=g_{y}(w_{y}a^{<t>}+b_{y})
\end{array}\right.$<br>
$w_{a}=[w_{ax},w_{aa}]\in\mathbb{R}^{h\times(d+h)}$ is the horizontal concatenation of $w_{ax}$ and $w_{aa}$. We have $w_{y}=w_{ya}$.

## Backpropagation through time (BPTT)/ Exploding gradient and vanishing gradient
**Backpropagation through time (BPTT)**<br>
Backpropagation through time (BPTT) is actually a specific application of backpropagation in RNNs. In modern deep learning frameworks, you only have to implement the forward pass, and the framework takes care of the backward pass. $w_{ax},w_{aa},w_{ya}$ do not change with time. In other words, $w_{ax},w_{aa},w_{ya}$ do not change across the layers.  

In this section, we consider an RNN without bias parameters. We suppose $T_{x}=T_{y}=T$. We also suppose $g_{a}=g$, $g_{y}$ is the identity function. Then, the RNN is gouverned by the following equations:<br>
$\left\{\begin{array}{l} 
z^{<t>}= w_{ax}X^{<t>}+w_{aa}a^{<t-1>}\\
a^{<t>}=g(w_{ax}X^{<t>}+w_{aa}a^{<t-1>})=g(z^{<t>})\\
\hat{Y}^{<t>}=w_{ya}a^{<t>}
\end{array}\right.$<br>


The loss at time step t is given by $\mathcal{L}^{<t>}=\mathcal{L}(\hat{Y}^{<t>},Y^{<t>})$, where $\mathcal{L}$ is a loss function. $\mathcal{L}$ could be 
* the least squares loss function : $\mathcal{L}(\hat{Y}^{<t>},Y^{<t>})=\frac{1}{2}\left(Y^{<t>}-\hat{Y}^{<t>}\right)^{2}$. 
* the cross-entropy loss function : $\mathcal{L}(\hat{Y}^{<t>},Y^{<t>})=-Y^{<t>}\log\left(\hat{Y}^{<t>}\right)-\left(1-Y^{<t>}\right)\log\left(1-\hat{Y}^{<t>}\right)$

The overall loss is given by $J=J(\hat{Y},Y)=\sum_{t=1}^{T}\mathcal{L}^{<t>}=\sum_{t=1}^{T}\mathcal{L}(\hat{Y}^{<t>},Y^{<t>})=J(w_{ax},w_{aa},w_{ya})$

We use backpropagation to train an RNN using the gradient of the overall loss J with respect to $w_{ax},w_{aa},w_{ya}$. In order to compute the gradient we need to compute $\frac{\partial J}{\partial w_{aa}}, \frac{\partial J}{\partial w_{ax}}, \frac{\partial J}{\partial w_{ya}}$. We have:
* $\frac{\partial J}{\partial w_{aa}}=\sum_{t=1}^{T}\frac{\partial \mathcal{L}^{<t>}}{\partial w_{aa}}=\sum_{t=1}^{T}\frac{\partial \mathcal{L}(\hat{Y}^{<t>},Y^{t})}{\partial w_{aa}}$
* $\frac{\partial J}{\partial w_{ax}}=\sum_{t=1}^{T}\frac{\partial \mathcal{L}^{<t>}}{\partial w_{ax}}=\sum_{t=1}^{T}\frac{\partial \mathcal{L}(\hat{Y}^{<t>},Y^{t})}{\partial w_{ax}}$
* $\frac{\partial J}{\partial w_{ya}}=\sum_{t=1}^{T}\frac{\partial \mathcal{L}^{<t>}}{\partial w_{ya}}=\sum_{t=1}^{T}\frac{\partial \mathcal{L}(\hat{Y}^{<t>},Y^{t})}{\partial w_{ya}}$

We have $\frac{\partial a^{<t>}}{\partial w_{aa}}=\frac{\partial g(z^{<t>})}{\partial w_{aa}}=\frac{\partial g(z^{<t>})}{\partial z^{<t>}}\frac{\partial z^{<t>}}{\partial w_{aa}}=g^{\prime}(z^{<t>})\frac{\partial (w_{ax}X^{<t>}+w_{aa}a^{<t-1>})}{\partial w_{aa}}=g^{\prime}(z^{<t>})\left[a^{<t-1>}+w_{aa}\frac{\partial a^{<t-1>}}{\partial w_{aa}} \right]$. Suppose you want to compute $\frac{\partial a^{<t-1>}}{\partial w_{aa}}$, you have to go back again (because we need to compute $\frac{\partial a^{<t-2>}}{\partial w_{aa}}$). This is why it is called backpropagation through time. We have:
* $\frac{\partial a^{<1>}}{\partial w_{aa}}=g^{\prime}(z^{<1>})a^{<0>}$ because $a^{<0>}$ does not depend on $w_{aa}$ 
* $\frac{\partial a^{<2>}}{\partial w_{aa}}=g^{\prime}(z^{<2>})\left[a^{<1>}+w_{aa}\frac{\partial a^{<1>}}{\partial w_{aa}} \right]$
* $\frac{\partial a^{<3>}}{\partial w_{aa}}=g^{\prime}(z^{<3>})\left[a^{<2>}+w_{aa}\frac{\partial a^{<2>}}{\partial w_{aa}} \right]$

The above equations lead to $$\frac{\partial a^{<3>}}{\partial w_{aa}}=g^{\prime}(z^{<3>})a^{<2>}+w_{aa}g^{\prime}(z^{<3>})g^{\prime}(z^{<2>})a^{<1>}+w^{2}_{aa}g^{\prime}(z^{<3>})g^{\prime}(z^{<2>})g^{\prime}(z^{<1>})a^{<0>}=\sum_{i=1}^{3}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<3-j>}) \right]a^{<3-i>}$$

Now we can assume that :
$$\frac{\partial a^{<t>}}{\partial w_{aa}}=\sum_{i=1}^{t}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<t-j>}) \right]a^{<t-i>}$$

In order to compute $\frac{\partial J}{\partial w_{aa}}$ we have to compute $\frac{\partial \mathcal{L}^{<t>}}{\partial w_{aa}}$ for each time step t:
$$\frac{\partial \mathcal{L}^{<t>}}{\partial w_{aa}}= \frac{\partial \mathcal{L}^{<t>}}{\partial \hat{Y}^{<t>}}\frac{\partial \hat{Y}^{<t>}}{\partial a^{<t>}}\frac{\partial a^{<t>}}{\partial w_{aa}}=\frac{\partial \mathcal{L}^{<t>}}{\partial \hat{Y}^{<t>}}w_{ya}\sum_{i=1}^{t}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<t-j>}) \right]a^{<t-i>}$$

let's have a look at $$\frac{\partial a^{<t>}}{\partial w_{ax}}=\frac{\partial g(z^{<t>})}{\partial w_{ax}}=g^{\prime}(z^{<t>})\frac{\partial z^{<t>}}{\partial w_{ax}}=g^{\prime}(z^{<t>})\frac{\partial (w_{ax}X^{<t>}+w_{aa}a^{<t-1>})}{\partial w_{ax}}=g^{\prime}(z^{<t>})\left[X^{<t>}+w_{aa}\frac{\partial a^{<t-1>}}{\partial w_{aa}} \right]$$

We have:
* $\frac{\partial a^{<1>}}{\partial w_{ax}}=g^{\prime}(z^{<1>})X^{<0>}$
* $\frac{\partial a^{<2>}}{\partial w_{ax}}=g^{\prime}(z^{<2>})\left[X^{<1>}+w_{aa}\frac{\partial a^{<1>}}{\partial w_{aa}}\right]$
* $\frac{\partial a^{<3>}}{\partial w_{ax}}=g^{\prime}(z^{<3>})\left[X^{<2>}+w_{aa}\frac{\partial a^{<2>}}{\partial w_{aa}}\right]$

then the above equations lead to:
$$\frac{\partial a^{<3>}}{\partial w_{ax}}=g^{\prime}(z^{<3>})X^{<2>}+w_{aa}g^{\prime}(z^{<3>})g^{\prime}(z^{<2>})X^{<1>}+w^{2}_{aa}g^{\prime}(z^{<3>})g^{\prime}(z^{<2>})g^{\prime}(z^{<1>})X^{<0>}=\sum_{i=1}^{3}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<3-j>}) \right]X^{<3-j>}$$

We can assume:
$$\frac{\partial a^{<t>}}{\partial w_{ax}}=\sum_{i=1}^{t}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<t-j>}) \right]X^{<t-j>} $$

In order to compute $\frac{\partial J}{\partial w_{ax}}$ we need to compute $\frac{\partial \mathcal{L}^{t}}{\partial w_{ax}}$ for each time step t.
$$\frac{\partial \mathcal{L}^{<t>}}{\partial w_{ax}}=\frac{\partial \mathcal{L}^{<t>}}{\partial \hat{Y}^{<t>}}\frac{\partial \hat{Y}^{<t>}}{\partial a^{<t>}}\frac{\partial a^{<t>}}{\partial w_{ax}}=\frac{\partial \mathcal{L}^{<t>}}{\partial \hat{Y}^{<t>}}w_{ya}\sum_{i=1}^{t}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<t-j>}) \right]X^{<t-j>} $$

In order to compute $\frac{\partial J}{\partial w_{ya}}$ we need to compute $\frac{\partial \mathcal{L}^{<t>}}{\partial w_{ya}}$ for each time step t.
$$\frac{\partial \mathcal{L}^{<t>}}{\partial w_{ya}}=\frac{\partial \mathcal{L}^{<t>}}{\partial Y^{<t>}} \frac{\partial \hat{Y}^{<t>}}{\partial w_{ya}}=\frac{\partial \mathcal{L}^{<t>}}{\partial Y^{<t>}}a^{<t>}$$

**Exploding gradient and vanishing gradient** <br>
Let $w$ be a weights matrix.
We are in the case of exploding gradient when $\left\lVert\frac{\partial J}{\partial w}\right\rVert\rightarrow +\infty$ during backprobagation. When $\left\lVert\frac{\partial J}{\partial w}\right\rVert\rightarrow 0$, this is called vanishing gradient.<br>
We saw in an earlier section that:
$$\frac{\partial \mathcal{L}^{<T>}}{\partial w_{aa}}=\frac{\partial \mathcal{L}^{<T>}}{\partial \hat{Y}^{<T>}}w_{ya}\sum_{i=1}^{T}w^{i-1}_{aa}\left[\prod_{j=0}^{i-1}g^{\prime}(z^{<T-j>}) \right]a^{<T-i>}$$
We can see $\frac{\partial \mathcal{L}^{<T>}}{\partial w_{aa}}$ already exhibits some key problems of long sequence models (T is very large): it involves potentially very large powers of $w_{aa}$. In it, eigenvalues smaller than 1 vanish and eigenvalues larger than 1 diverge. This is numerically unstable, which manifests itself in the form of vanishing and exploding gradients.<br>
To avoid exploding gradient, we simply use a method called gradient clipping where at each timestamp, we can check if the gradient > threshold and if it is, we normalize it. This helps to tackle exploding gradient. To tackle the vanishing gradient problem these are the possible solutions:
* Use ReLU instead of tanh or sigmoid activation function
* Proper initialization of the weight matrices can reduce the effect of vanishing gradients. It has been seen that initializing with an identity matrix helps in tackling this problem.
* Using gated cells such as LSTM or GRUs

## CONCLUSIONS ABOUT RNNs
* RNNs are slow to train
* Long sequences leed to vanishing gradient problem and exploding gradients problems
* Input data needs to be passed sequentially or serially one after the other : such sequential flow does not make use of todays GPUs very well which are designed for parallel computation

# Types of recurrent neural networks
http://datahacker.rs/003-rnn-architectural-types-of-different-recurrent-neural-networks/

So far we've seen an RNN architecture where the number of inputs $T_{x}$ is equal to the number of outputs $T_{y}$. It turns out that for other applications $T_{x}$ and $T_{y}$ may always be the same. The RNN architecture depends on the problem we want to handle.<br>
Different types of RNNs are used for different use cases, such as music generation, name entity recognition, sentiment classification, and machine translation.<br>
In the example of music generation, $T_{x}$ can be equal to 1 and $T_{y}$ is greater than 1. In sentiment classification the output $Y$ could be an integer from 1 to 4 wheras the input is a sequence. In name entity recognition the input length $T_{x}$ and output length $T_{y}$ are identical. In machine translation the input and ouput have different lengths.

the four major categories of RNN architectures are:
* One-to-one : A single input that predicts a single output forms what we call a One-to-One architecture. 
* One-to-many : A single input that results in multiple output values or an output sequence is called a One-to-Many architecture. This particular architecture can be found in the Music Generation problems.
* Many-to-one : We will take the case of Sentiment Classification to explain this category of RNN models where there are multiple inputs but only one output value.
* Many-to-many : Machine Translation is one such real-life case where the input sentence says, in French, and the output sentence is a translated English one, which may have a different number of words than the original sentence but of course, the meaning remains the same.

<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/RNN%20ARCHITECTURE.jpg" width="800" height="400">


> 

# GRU :Gated Recurrent Unit
https://blog.floydhub.com/gru-with-pytorch/

A Gated Recurrent Unit (GRU), as its name suggests, is a variant of the RNN architecture, and uses gating mechanisms to control and manage the flow of information between cells in the neural network.<br>
The structure of the GRU allows it to adaptively capture dependencies from large sequences of data without discarding information from earlier parts of the sequence. This is achieved through its gating units, which solve the vanishing/exploding gradient problem of traditional RNNs. These gates are responsible for regulating the information to be kept or discarded at each time step.

A GRU cell at time step t is governed by the following equations:<br>
$\left\{\begin{array}{l} 
\Gamma_{u} = \sigma(w_{u}[c^{<t-1>},X^{<t>}]^{t}+b_{u})\\
\Gamma_{r} = \sigma(w_{r}[c^{<t-1>},X^{<t>}]^{t}+b_{r})\\
\overset{\sim}{c}^{<t>} = tanh(w_{c}[\Gamma_{r}*c^{<t-1>},X^{<t>}]^{t}+b_{c})\\
c^{<t>} = \Gamma_{u}*\overset{\sim}{c}^{<t>} + (1-\Gamma_{u})*c^{<t-1>}\\
c^{<t>} = a^{<t>}\\
\hat{Y}^{<t>}=g_{y}(w_{yc}c^{<t>}+b_{y})
\end{array}\right.$<br>
$*$ is the the element-wise product.

$\Gamma_{u}=\sigma(w_{u}[c^{<t-1>},X^{<t>}]+b_{u})$ is the update gate. $\Gamma_{r}=\sigma(w_{r}[c^{<t-1>},X^{<t>}]+b_{r})$ is the reset gate. $\sigma$ is the sigmoid function. The sigmoid function has values alwayse between 0 and 1. $\overset{\sim}{c}^{<t>} = tanh(w_{c}[\Gamma_{r}*c^{<t-1>},X^{<t>}]+b_{c})$ is the new proposed hidden state. $c^{<t>} = \Gamma_{u}*\overset{\sim}{c}^{<t>} + (1-\Gamma_{u})*c^{<t-1>}$ is the final updated hidden state.<br>
* The update gate $\Gamma_{u}\in\mathbb{R}^{h\times n}$. The bias $b_{u}\in\mathbb{R}^{h\times 1}$. The weight parameter $w_{u}\in\mathbb{R}^{d(h+d)}$
* The reset gate $\Gamma_{r}\in\mathbb{R}^{h\times n}$. The bias $b_{r}\in\mathbb{R}^{h\times 1}$. The weight parameter $w_{r}\in\mathbb{R}^{d(h+d)}$
* The candidate hidden state $\overset{\sim}{c}^{<t>}\in\mathbb{R}^{h\times n}$. The weight parameter $w_{c}\in\mathbb{R}^{d(h+d)}$. The bias $b_{c}\in\mathbb{R}^{h\times 1}$. The hidden state $c^{<t>}\in\mathbb{R}^{h\times n}$.
* The output $\hat{Y}^{<t>}\in \mathbb{R}^{q\times n}$. The weight parameter $w_{yc}\in \mathbb{R}^{q\times h}$. $b_{y}\in \mathbb{R}^{q\times 1}$ is the bias parameter.

The Update gate and the Reset gates in the GRU are trained to selectively filter out any irrelevant information while keeping what’s useful. These gates are essentially vectors containing values between 0 to 1 which will be multiplied with the input data and/or hidden state. A 0 value in the gate vectors indicates that the corresponding data in the input or hidden state is unimportant and will, therefore, return as a zero. On the other hand, a 1 value in the gate vector means that the corresponding data is important and will be used.<br><br>
The Reset gate is responsible for deciding which portions of the previous hidden state are to be combined with the current input to propose a new hidden state.<br>
The Update gate is responsible for determining how much of the previous hidden state is to be retained and what portion of the new proposed hidden state (derived from the Reset gate) is to be added to the final hidden state. Below, we draw the inner workings of the GRU cell.

Below, we draw the inner workings of the GRU cell.


<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/Inner_workings_gru_cell.png" width="100%" />
</center>

Now, we draw the overall GRU network.
<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/overall_gru_network.png" width="100%" />
</center>

# LSTM : Long short-term memory
https://blog.floydhub.com/gru-with-pytorch/<br>
https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/<br>
https://blog.mlreview.com/understanding-lstm-and-its-diagrams-37e2f46f1714<br>
https://www.analyticsvidhya.com/blog/2021/03/introduction-to-long-short-term-memory-lstm/

Long Short Term Memory Network (LSTM) is an advanced RNN, a sequential network, that allows information to persist. LSTM is capable of handling long-term dependencies.
LSTM uses gating mechanism within each LSTM cell. LSTM is capable of handling the vanishing gradient problem faced by RNN. LSTM still faces the issue of Exploding Gradient <br>
Like a GRU, LSTM allow us to learn very long range dependencies and connections.
In other words, LSTMs have the ability to preserve long-term memory. This is especially important in the majority of Natural Language Processing (NLP) or time-series and sequential tasks.<br>
LSTMs have a complex structure. At each time step, the LSTM cell takes in 3 different pieces of information -- the current input data, the short-term memory from the previous cell (similar to hidden states in RNNs) and lastly the long-term memory.<br>
The short-term memory is commonly referred to as the hidden state, and the long-term memory is usually known as the cell state.

The cell then uses gates to regulate the information to be kept or discarded at each time step before passing on the long-term and short-term information to the next cell.<br>
Of course, these gates need to be trained to accurately filter what is useful and what is not.

A LSTM cell at time step t is governed by the following equations:<br>

$\left\{\begin{array}{l} 
\overset{\sim}{c}^{<t>} = tanh(w_{c}[a^{<t-1>},X^{<t>}]^{t}+b_{c})\\
\Gamma_{u}=\sigma(w_{u}[a^{<t-1>},X^{<t>}]^{t}+b_{u})\\
\Gamma_{f}=\sigma(w_{f}[a^{<t-1>},X^{<t>}]^{t}+b_{f})\\
\Gamma_{o}=\sigma(w_{o}[a^{<t-1>},X^{<t>}]^{t}+b_{o})\\
c^{<t>} = \Gamma_{u}*\overset{\sim}{c}^{<t>}+\Gamma_{f}*c^{<t-1>}\\
a^{<t>} = \Gamma_{o}*tanh(c^{<t>})\\
\hat{Y}^{<t>}=g_{y}(w_{ya}a^{<t>}+b_{y})
\end{array}\right.$<br>
$*$ is the element-wise product.

* $c^{<t-1>}\in\mathbb{R}^{h\times n}$ is the previous long-term memory/cell state/memory cell.
* $c^{<t>}\in\mathbb{R}^{h\times n}$ is the new/updated long-term memory/cell state/memory cell.
* $a^{<t-1>}\in\mathbb{R}^{h\times n}$ is the previous short-term memory/hidden state.
* $a^{<t>}\in\mathbb{R}^{h\times n}$ is the new short-term memory/hidden state.
* $\overset{\sim}{c}^{<t>}$ is the candidate value for the new cell state $c^{<t>}$.
* $\Gamma_{u}\in\mathbb{R}^{h\times n}$ is the update gate. $\Gamma_{f}\in\mathbb{R}^{h\times n}$ is the forget gate. $\Gamma_{o}\in\mathbb{R}^{h\times n}$ is the output gate.
* $w_{c},w_{u},w_{f},w_{o}\in\mathbb{R}^{d(d+h)}$ are weight parameters. $b_{c},b_{u},b_{f},b_{o}\in\mathbb{R}^{h\times 1}$.
* The output $\hat{Y}^{<t>}\in \mathbb{R}^{q\times n}$. The weight parameter $w_{ya}\in \mathbb{R}^{q\times h}$. $b_{y}\in \mathbb{R}^{q\times 1}$ is the bias parameter.

Below, we draw the inner workings of the LSTM cell. Below, the diagram of a LSTM building block:

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/LSTM_CELL_ARCHITECTURE.png" width="90%" />
</center>

Now, we draw the overall LSTM network. 
<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/overall_LSTM_network.png" width="120%" />
</center>

# Bidirectional recurrent neural networks (BRNN)
So far, we dealt with unidirectional RNN that passes information in a forward direction.
Instead of running an RNN only in the forward mode starting from the first input, we start another one from the last input running from back to front. Bidirectional RNNs add a hidden layer that passes information in a backward direction.<br>
BRNN lets you at a point time to take imformation from both earlier and later in the sequence. In this section we deal with Bidirectional standart RNN, Bidirectional GRU and Bidirectional LSTM.

## Bidirectional standart RNN
<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/BIDIRECTIONAL_RNN_network.png" width="100%" />
</center>


A Bidirectional standart RNN is governed by the following equations:<br>
$\left\{\begin{array}{l} 
\overrightarrow{a}^{<t>}=g_{a}^{(f)}(w_{a}^{(f)}[X^{<t>},\overrightarrow{a}^{<t-1>}]+b_{a}^{(f)})\\
\overleftarrow{a}^{<t>}=g_{a}^{(b)}(w_{a}^{(b)}[X^{<t>},\overleftarrow{a}^{<t+1>}]+b_{a}^{(b)})\\
\hat{Y}^{<t>}=g_{y}(w_{y}[\overrightarrow{a}^{<t>},\overleftarrow{a}^{<t>}]+b_{y})
\end{array}\right.$<br>
$\overrightarrow{a}^{<t>}$ is the forward hidden state and $\overleftarrow{a}^{<t>}$ is the backward hidden state. In order to compute $\hat{Y}^{<t>}$ we concatenate $\overrightarrow{a}^{<t>}$ and $\overleftarrow{a}^{<t>}$.

## Bidirectional GRU
<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/BIDIRECTIONAL_GRU_network.png" width="90%" />
</center>

A Bidirectional GRU is governed by the following equations:<br>
**Forward equations:**<br>
$\left\{\begin{array}{l} 
\overrightarrow{\Gamma}_{u} = \sigma(w_{u}^{(f)}[\overrightarrow{c}^{<t-1>},X^{<t>}]+b_{u}^{(f)})\\
\overrightarrow{\Gamma}_{r} = \sigma(w_{r}^{(f)}[\overrightarrow{c}^{<t-1>},X^{<t>}]+b_{r}^{(f)})\\
\overrightarrow{\overset{\sim}{c}}^{<t>} = tanh(w_{c}^{(f)}[\overrightarrow{\Gamma_{r}}*\overrightarrow{c}^{<t-1>},X^{<t>}]+b_{c}^{(f)})\\
\overrightarrow{c}^{<t>} = \overrightarrow{\Gamma}_{u}*\overrightarrow{\overset{\sim}{c}}^{<t>} + (1-\overrightarrow{\Gamma}_{u})*\overrightarrow{c}^{<t-1>}\\
\overrightarrow{c}^{<t>} = \overrightarrow{a}^{<t>}
\end{array}\right.$<br>

**Backward equations:**<br>
$\left\{\begin{array}{l} 
\overleftarrow{\Gamma}_{u} = \sigma(w_{u}^{(b)}[\overleftarrow{c}^{<t+1>},X^{<t>}]+b_{u}^{(b)})\\
\overleftarrow{\Gamma}_{r} = \sigma(w_{r}^{(b)}[\overleftarrow{c}^{<t+1>},X^{<t>}]+b_{r}^{(b)})\\
\overleftarrow{\overset{\sim}{c}}^{<t>} = tanh(w_{c}^{(b)}[\overleftarrow{\Gamma_{r}}*\overleftarrow{c}^{<t+1>},X^{<t>}]+b_{c}^{(b)})\\
\overleftarrow{c}^{<t>} = \overleftarrow{\Gamma}_{u}*\overleftarrow{\overset{\sim}{c}}^{<t>} + (1-\overleftarrow{\Gamma}_{u})*\overleftarrow{c}^{<t+1>}\\
\overleftarrow{c}^{<t>} = \overleftarrow{a}^{<t>}
\end{array}\right.$<br>
 we concatenate the forward $\overrightarrow{c}^{<t>}$ and backward hidden states $\overleftarrow{c}^{<t>}$ and we compute $\hat{Y}^{<t>}=g_{y}(w_{y}[\overrightarrow{c}^{<t>},\overleftarrow{c}^{<t>}]+b_{y})$

##  Bidirectional LSTM

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/BIDIRECTIONAL_LSTM_network.png" width="100%" />
</center>

A bidirectional LSTM is governed by the following equations:<br>
**forward equations:**<br>
$\left\{\begin{array}{l} 
\overrightarrow{\overset{\sim}{c}}^{<t>} = tanh(w_{c}^{(f)}[\overrightarrow{a}^{<t-1>},X^{<t>}]+b_{c}^{(f)})\\
\overrightarrow{\Gamma}_{u}=\sigma(w_{u}^{(f)}[\overrightarrow{a}^{<t-1>},X^{<t>}]+b_{u}^{(f)})\\
\overrightarrow{\Gamma}_{f}=\sigma(w_{f}^{(f)}[\overrightarrow{a}^{<t-1>},X^{<t>}]+b_{f}^{(f)})\\
\overrightarrow{\Gamma}_{o}=\sigma(w_{o}^{(f)}[\overrightarrow{a}^{<t-1>},X^{<t>}]+b_{o}^{(f)})\\
\overrightarrow{c}^{<t>} = \overrightarrow{\Gamma}_{u}*\overrightarrow{\overset{\sim}{c}}^{<t>}+\overrightarrow{\Gamma}_{f}*\overrightarrow{c}^{<t-1>}\\
\overrightarrow{a}^{<t>} = \overrightarrow{\Gamma}_{o}*tanh(\overrightarrow{c}^{<t>})
\end{array}\right.$<br>
$*$ is the element-wise product.

**backward equations:**<br>
$\left\{\begin{array}{l} 
\overleftarrow{\overset{\sim}{c}}^{<t>} = tanh(w_{c}^{(b)}[\overleftarrow{a}^{<t+1>},X^{<t>}]+b_{c}^{(b)})\\
\overleftarrow{\Gamma}_{u}=\sigma(w_{u}^{(b)}[\overleftarrow{a}^{<t+1>},X^{<t>}]+b_{u}^{(b)})\\
\overleftarrow{\Gamma}_{f}=\sigma(w_{f}^{(b)}[\overleftarrow{a}^{<t+1>},X^{<t>}]+b_{f}^{(b)})\\
\overleftarrow{\Gamma}_{o}=\sigma(w_{o}^{(b)}[\overleftarrow{a}^{<t+1>},X^{<t>}]+b_{o}^{(b)})\\
\overleftarrow{c}^{<t>} = \overleftarrow{\Gamma}_{u}*\overleftarrow{\overset{\sim}{c}}^{<t>}+\overleftarrow{\Gamma}_{f}*\overleftarrow{c}^{<t+1>}\\
\overleftarrow{a}^{<t>} = \overleftarrow{\Gamma}_{o}*tanh(\overleftarrow{c}^{<t>})
\end{array}\right.$<br>

we concatenate the forward $\overrightarrow{a}^{<t>}$ and backward hidden states $\overleftarrow{a}^{<t>}$ and we compute $\hat{Y}^{<t>}=g_{y}(w_{ya}[\overrightarrow{a}^{<t>},\overleftarrow{a}^{<t>}]+b_{y})$

# Sequence-to-Sequence Models. Encoder-decoder 
Sequence-to-sequence (seq2seq) models in NLP are used to convert sequences of Type A to sequences of Type B. For example, translation of French sentences to English sentences is a sequence-to-sequence task.<br>
Speech recognition is a sequence-to-sequence problem : the input sequence is an audio clip. The output sequence is the Audio Transcription.

Let’s take a simple example of a sequence-to-sequence model designed to translate from french to english. Check out the below illustration:

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/Encoder_Decoder_network.png" width="100%" />
</center>

A seq2seq model uses an encoder network to make an encoding of the input french sentence. In a second step, the seq2seq model passes the encoding of the french input sentence (known as the context vector) to the decoder network in order to generate the corresponding English translation.<br>
We can consider machine translation as building a conditional language model. The decoder network is very close to a langage model.<br>
The seq2seq model computs :
$$\hat{Y}^{<1>},\dots,\hat{Y}^{<T_{y}>}=\underset{Y^{<1>},\dots,Y^{<T_{y}>}}{\mathrm{argmax}} P(Y^{<1>},\dots,Y^{<T_{y}>}|X^{<1>},\dots,X^{<T_{x}>}) $$
where $P(\overbrace{Y^{<1>},\dots,Y^{<T_{y}>}}^{English}|\overbrace{X^{<1>},\dots,X^{<T_{x}>}}^{French})$ is the conditional probability of an english translation given an input french sentence.<br>
The seq2seq model yieds $\hat{Y}^{<1>},\dots,\hat{Y}^{<T_{y}>}$ as an english translation of the input french sentence.
* Both Encoder and Decoder are LSTMs
* At every time step in the Encoder, the LSTM takes a word vector $X^{<i>}$ from the input sequence and a hidden state $a^{<i-1>}$ from the previous time step
* The hidden state is updated at each time step
* The hidden state from the last unit is known as the **context vector**. This contains information about the input sequence
* This context vector is then passed to the decoder and it is then used to generate the target sequence (English phrase)


There are certain limitations of seq-2-seq models:
* The encoder-decoder architecture works quite well for short sentences but for very long sentences the performance comes down. This challenge is addressed by the attention model.
* The sequential nature of the RNN architecture prevents parallelization. These challenges are addressed by the transformers. 

# Attention Mechanism using LSTM/RNN

In this section, we describe the attention model in the context of machine translation from french to english.  We can also apply the attention mechanism to other problems such as image captioning.<br>
In the context of machine translation we saw in the above section that the performance of the encoder-decoder network degrades rapidly as the length of the input sentence increases.<br>
Another problem is that there is no way to give more importance to some of the input words compared to others while translating the sentence. The attention mechanism tells a Neural Machine Translation model where it should pay attention to at any step.

There are 3 parts in the attention model: pre-attention part, attention part and the post-attention part.

The pre-attention part is a Bidirectional LSTM that takes the $T_{x}$ word vectors $(X^{<1>},....,X^{<T_{x}>})$ of the french sentence as an input. The Bidirectional LSTM is called pre-attention Bi-LSTM. Then The Bidirectional LSTM generates a sequence of annotations (the hidden state vectores) $(a^{<1>}=[\overrightarrow{a}^{<1>},\overleftarrow{a}^{<1>}], a^{<2>}=[\overrightarrow{a}^{<2>},\overleftarrow{a}^{<2>}],\dots, a^{<T_{x}>}=[\overrightarrow{a}^{<T_{x}>},\overleftarrow{a}^{<T_{x}>}])$. $a^{<t>}=[\overrightarrow{a}^{<t>},\overleftarrow{a}^{<t>}]$ represents the concatenation of the activations of both the forward-direction and backward-directions of the pre-attention Bi-LSTM.

The post-attention part is a basic RNN called post-attention RNN. The post-attention RNN generates a sequence $(\hat{Y}^{<1>},\dots,\hat{Y}^{<T_{y}>})$ of length $T_{y}$ corresponding to the english translation. We call this RNN post-attention RNN. We use $S^{<t>}$ to denote the hidden state at time step t of the post-attention RNN. The input at time step t of the post-attention RNN is $$c^{<t>}=\sum_{t^{\prime}=1}^{T_{x}}\alpha^{<t,t^{\prime}>}a^{<t^{\prime}>}$$
We call $c^{<t>}$ the context vector at time step t. We have $\sum_{t^{\prime}=1}^{T_{x}}\alpha^{<t,t^{\prime}>}=1$ and $\alpha^{<t,t^{\prime}>}\geq 0$. $[\alpha^{<t,1>},\dots,\alpha^{<t,T_{x}}>]$ are the attention weights at time step t. $\alpha^{<t,t^{\prime}>}$ is the amount of attention $Y^{<t>}$ should pay $a^{<t^{\prime}>}$ when we trying to generate the tth words of the english translation. 

the attention part generats the attention weights. The attention part is composed of $T_{y}$ elementary attention blocks. The tth elementary attention block generats $[\alpha^{<t,1>},\dots,\alpha^{<t,T_{x}}>]$, the attention weights related to the tth context vector. The tth attention block takes the hidden states $a^{<1>},\dots,a^{<T_{x}>} $ of the pre-atention LSTM and the previous hidden state $S^{<t-1>}$ of the post attention RNN as an input.<br>
Then the tth attention block passes $S^{<t-1>}$ and $a^{<t^{\prime}>}$ to a dense layer to compute $e^{<t,t^{\prime}>}$ for $t^{\prime} \in \{1,\dots,T_{x}\}$. Then we passe $(e^{<t,1>}),\dots,e^{<t,T_{x}>})$ to a softmax function in order to compute the attention weights:
$$\alpha^{<t,t^{\prime}>}=\frac{\exp(e^{<t,t^{\prime}>})}{\sum_{t=1}^{T_{x}}\exp(e^{<t,t^{\prime}>})} $$
Check out the below illustration to understand the inner workings of an elementary attention block:

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/attn_mechanism.png" width="400" height="400" >
</center>

Below, the overall attention model:

<center>
<img src="https://raw.githubusercontent.com/fabnancyuhp/DEEP-LEARNING/main/IMAGE/RNN/attn_model.png" width="450" height="450">
</center>